In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob, os
import pandas as pd
from dask import delayed, compute
from dask.distributed import Client, LocalCluster, performance_report
from solardatatools import DataHandler
from dask_cloudprovider.aws import FargateCluster

import sys
sys.path.append('../..')
from sdt_dask.dask_tool.sdt_dask import SDTDask
from sdt_dask.dataplugs.pvdaq_plug import PVDAQPlug
from sdt_dask.dataplugs.csv_plug import LocalFiles

In [3]:
SDTDask?

Init signature: SDTDask(data_plug, client, **keywords)
Docstring:      <no docstring>
File:           /mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/sdt_dask/dask_tool/sdt_dask.py
Type:           type
Subclasses:     

In [4]:
PVDAQPlug?

Init signature: PVDAQPlug(api_key='DEMO_KEY', power_col='ac_power')
Docstring:     
Dataplug class for retrieving data from the PVDAQ DB.
Note that the DEMO_KEY has a rate limit of 30/h, 50/d per IP address.
File:           /mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/sdt_dask/dataplugs/pvdaq_plug.py
Type:           type
Subclasses:     

In [5]:
LocalFiles?

Init signature: LocalFiles(path_to_files, ext='.csv')
Docstring:     
Dataplug class for retrieving data from some source. It's recommended
that user-created dataplug inherit from this class to ensure compatibility.

The initialization argument for each class will be different depending on
the source. The main requirement is to keep the ``Dataplug.get_data`` method,
and make sure the args and returns as defined here.
File:           /mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/sdt_dask/dataplugs/csv_plug.py
Type:           type
Subclasses:     

# Instantiate a pvdaq data plug


In [6]:
pvdaq_data_plug = PVDAQPlug()

# Instantiate a localfile data plug

In [7]:
path = "../dataplugs/example_data/"
local_file_data_plug = LocalFiles(path_to_files=path)
local_file_KEYS = [os.path.basename(fname)[:-4] for fname in glob.glob(path + "*")]
local_file_KEYS = local_file_KEYS[1:]
local_file_KEYS

['TAAI01129193', 'TABC01142170', 'TABD01110568']

In [8]:
local_file_data_plug.get_data(local_file_KEYS[1])

Loading file TABC01142170...


,ac_power_01,ac_power_02,ac_power_03
ts,,,
2013-12-31 16:00:00,0.0767,0.0928,0.0928
2013-12-31 16:05:00,0.0695,0.0862,0.0819
2013-12-31 16:10:00,0.0626,0.0726,0.0653
2013-12-31 16:15:00,0.0546,0.0639,0.0613
2013-12-31 16:20:00,0.0418,0.0530,0.0516
...,...,...,...
2019-06-20 15:30:00,0.5872,0.5950,0.2445
2019-06-20 15:35:00,0.6454,0.6506,0.2748
2019-06-20 15:40:00,0.6220,0.6313,0.2645


# Set up a Fargate cluster client


In [9]:
# Set up
PA_NUMBER = os.getenv("project_pa_number")
TAGS = {
    "project-pa-number": PA_NUMBER,
    "project": "pvinsight"
}

IMAGE = "nimishy/sdt-dask-windows:latest"

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION = os.getenv("AWS_DEFAULT_REGION")
VPC = "vpc-ab2ff6d3" # for us-west-2

# Instantiate Fargate cluster
cluster = FargateCluster(
    tags=TAGS,
    image=IMAGE,
    vpc=VPC,
    region_name=AWS_DEFAULT_REGION,
    n_workers=3,
    worker_nthreads=1,
    environment={
        'AWS_ACCESS_KEY_ID': AWS_ACCESS_KEY_ID,
        'AWS_SECRET_ACCESS_KEY': AWS_SECRET_ACCESS_KEY
    }
)


fargate_client = Client(cluster)

KeyboardInterrupt: 

# Set a local User-defined Client

In [10]:
n_workers = 1
threads_per_worker = 1
total_system_memory = 8
memory_per_worker = total_system_memory / n_workers
local_client = Client(processes=False, memory_spill_fraction=False, memory_pause_fraction=False, memory_target_fraction=0.8, n_workers=n_workers, threads_per_worker=threads_per_worker, memory_limit=f"{memory_per_worker}GiB")
local_client.dashboard_link

2024-03-12 16:56:26,760 - distributed.worker.memory - WARNING - Ignoring provided memory limit 8.0GiB due to system memory limit of 7.66 GiB
/home/zhang/miniconda3/envs/manual-env/lib/python3.10/site-packages/distributed/worker_memory.py:494: FutureWarning: Parameter memory_target_fraction has been deprecated and will be removed in a future version; please use dask config key distributed.worker.memory.target instead
  warnings.warn(
/home/zhang/miniconda3/envs/manual-env/lib/python3.10/site-packages/distributed/worker_memory.py:494: FutureWarning: Parameter memory_spill_fraction has been deprecated and will be removed in a future version; please use dask config key distributed.worker.memory.spill instead
  warnings.warn(
/home/zhang/miniconda3/envs/manual-env/lib/python3.10/site-packages/distributed/worker_memory.py:494: FutureWarning: Parameter memory_pause_fraction has been deprecated and will be removed in a future version; please use dask config key distributed.worker.memory.pause 

'http://172.21.212.11:8787/status'

[============================================================] 100.0% ...queries complete in 2.0 seconds       



2024-03-12 16:56:31,581 - distributed.worker - WARNING - Compute Failed
Key:       run_pipeline-d1a46917-c79f-4021-982a-fe2554e904c8
Function:  execute_task
args:      ((<function apply at 0x7ffb9160f5b0>, <function run_pipeline at 0x7ffbd40ad870>, [<solardatatools.data_handler.DataHandler object at 0x7ffb6dc059c0>], (<class 'dict'>, [['solver', 'OSQP'], ['solver_convex', 'OSQP'], ['fix_shifts', True], ['verbose', True]])))
kwargs:    {}
Exception: 'TypeError("run_pipeline() got an unexpected keyword argument \'fix_shifts\'")'

2024-03-12 16:56:31,597 - distributed.worker - WARNING - Compute Failed
Key:       run_pipeline-5b75dea4-0162-4b23-b70e-0b7bd6364cf7
Function:  execute_task
args:      ((<function apply at 0x7ffb9160f5b0>, <function run_pipeline at 0x7ffbd40ad870>, [<solardatatools.data_handler.DataHandler object at 0x7ffb6d530a60>], (<class 'dict'>, [['solver', 'OSQP'], ['solver_convex', 'OSQP'], ['fix_shifts', True], ['verbose', True]])))
kwargs:    {}
Exception: 'TypeError("r



[============================================================] 100.0% ...queries complete in 1.7 seconds       

[============================================================] 100.0% ...queries complete in 1.5 seconds       

total time: 94.46 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              0.59s
Cleaning                   0.98s
Filtering/Summarizing      92.89s
    Data quality           0.06s
    Clear day detect       0.51s
    Clipping detect        91.96s
    Capacity change detect 0.35s

total time: 10.32 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              3.89s
Cleaning                   1.89s
Filtering/Summarizing      4.54s
    Data quality           0.12s
    Clear day detect       0.41s
    Clipping detect        3.60s
    Capacity change detect 0.42s

[============================================================] 100.0% ...queries complete in 2.2 secon

/home/zhang/miniconda3/envs/manual-env/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


total time: 12.38 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              5.62s
Cleaning                   1.87s
Filtering/Summarizing      4.88s
    Data quality           0.08s
    Clear day detect       0.60s
    Clipping detect        3.52s
    Capacity change detect 0.68s



# Use execute to get the result

## example 1
 use PVDAQ dataplug and local client

In [12]:
dask_tool = SDTDask(pvdaq_data_plug, local_client)

dask_tool.execute([(34, 2011), (35, 2015), (51,2012)])

In [ ]:
dask_tool = SDTDask(local_file_data_plug, local_client)

dask_tool.execute(local_file_KEYS)

## example 2
use LocalFile dataplug and fargate client

In [ ]:
dask_tool = SDTDask(local_file_data_plug, fargate_client)

dask_tool.execute(local_file_KEYS)